In [1]:
from data_utils import *
import IPython.display as ipd
import torch
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
hps = utils.get_hparams_from_file("configs/multispeaker.json")
import matplotlib.pyplot as plt

net_g = SynthesizerTrn(
  len(symbols),
  hps.data.filter_length // 2 + 1,
  hps.data.hop_length,
  hps.data.sampling_rate,
  hps.train.segment_size // hps.data.hop_length,
  n_speakers=hps.data.n_speakers,
  **hps.model)


DEBUG:matplotlib:matplotlib data path: /Users/xingyijin/.conda/envs/PycharmParent/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/Users/xingyijin/.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is darwin
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', '_heapq', 'heapq', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', 'collections.abc', '_functools', 'functools', 'contextli

In [2]:
def get_text(text):
    text_norm = text_to_sequence(text)
    text_norm = torch.LongTensor(text_norm)
    return text_norm
def tts(txt,sid=0,device="cpu"):
#     emo = emo *0
    stn_tst = get_text(txt)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        spk = torch.LongTensor([sid])
        t1 = time.time()
        res = net_g.to(device).infer(x_tst.to(device), x_tst_lengths.to(device),sid=spk, noise_scale=.667, noise_scale_w=0.8, length_scale=1)
        audio = res[0][0,0].cpu().data.float().numpy()
        f0 = res
        t2 = time.time()
        print("推理时间：", (t2-t1),"s")
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
    return f0[-1]

In [4]:
!ls logs_rewrite/ms/G_*

zsh:1: no matches found: logs_rewrite/ms/G_*


In [8]:
_ = utils.load_checkpoint("logs_rewrite/ms/G_26400.pth", net_g, None)

INFO:root:Loaded checkpoint 'logs_rewrite/ms/G_26400.pth' (iteration 34)


In [4]:
text = "志云大师急需扛起大旗收拾烂摊子."
f0 = tts(text, 0)
plt.plot(f0[0,:])
